In [1]:
!pwd
!ls
%cd "/content/drive/MyDrive/TC2008B/Reto/"
!pwd
%pip install mesa
%pip install pyngrok --quiet

/content
drive  sample_data
/content/drive/MyDrive/TC2008B/Reto
/content/drive/MyDrive/TC2008B/Reto


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# from retoagentes import Habitacion
#%cd "/content/drive/MyDrive/TC2008B/Reto/"
#from retoagentes import Habitacion

from retoagentes1 import Habitacion

import time
import datetime

# Install pyngrok to propagate the http server


# Load the required packages
from pyngrok import ngrok
from http.server import BaseHTTPRequestHandler, HTTPServer
import logging
import json
import os
import numpy as np

# Boid is a dummy implementation that can be replaced with a more sophisticated 
# MAS module, for example, MESA
#from boid import Boid

# Start ngrok
ngrok.install_ngrok()

# Terminate open tunnels if exist
ngrok.kill()

# Open an HTTPs tunnel on port 8585 for http://localhost:8585
port = os.environ.get("PORT", 8585)
server_address = ("", port)

#para unity
public_url = ngrok.connect(port="8585", proto="http", options={"bind_tls": True})
print("\n" + "#" * 94)
print(f"## Tracking URL: {public_url} ##")
print("#" * 94, end="\n\n")

ngrok.kill()

#####################################################################################################
tiempo_maximo = 0.05
ticks = 5

model = Habitacion(ticks)

#####################################################################################################

# The way how agents are updated (per step/iteration)
def updateFeatures():
    features = []

    # For each agent...
    
    #conti = conti + 1
    model.step()
    features = model.infoAgentes()
    #model.datacollector.get_model_vars_dataframe()

    return features

# Post the information in `features` for each iteration
def featuresToJSON(info_list):
    #import pdb; pdb.set_trace()
    featureDICT = []
    print(info_list)
    for info in info_list:
        feature = {
            "position" : info['position'], # position
            "kind" : info['kind'], # kind
            "colour" : info['colour'] # colour
        }
        featureDICT.append(feature)
    return json.dumps(featureDICT)

# This is the server. It controls the simulation.
# Server run (do not change it)
class Server(BaseHTTPRequestHandler):
    
    def _set_response(self):
        self.send_response(200)
        self.send_header('Content-type', 'text/html')
        self.end_headers()
        
    def do_GET(self):
        logging.info("GET request,\nPath: %s\nHeaders:\n%s\n", 
                     str(self.path), str(self.headers))
        self._set_response()
        self.wfile.write("GET request for {}".format(self.path).encode('utf-8'))

    def do_POST(self):
        content_length = int(self.headers['Content-Length'])

        #post_data = self.rfile.read(content_length)
        post_data = json.loads(self.rfile.read(content_length))
        
        # If you have issues with the encoder, toggle the following lines: 
        #logging.info("POST request,\nPath: %s\nHeaders:\n%s\n\nBody:\n%s\n",
                     #str(self.path), str(self.headers), post_data.decode('utf-8'))
        logging.info("POST request,\nPath: %s\nHeaders:\n%s\n\nBody:\n%s\n",
                     str(self.path), str(self.headers), json.dumps(post_data))

        # Here, magick happens 
        # --------------------       
        features = updateFeatures()
        #print(features)

        self._set_response()
        resp = "{\"data\":" + featuresToJSON(features) + "}"
        #print(resp)

        self.wfile.write(resp.encode('utf-8'))

# Server run (do not change it)
def run(server_class=HTTPServer, handler_class=Server, port=8585):
    logging.basicConfig(level=logging.INFO)
    server_address = ('', port)
    httpd = server_class(server_address, handler_class)

    public_url = ngrok.connect(port).public_url
    logging.info("ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(
        public_url, port))

    logging.info("Starting httpd...\n") # HTTPD is HTTP Daemon!
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:   # CTRL + C stops the server
        pass

    httpd.server_close()
    logging.info("Stopping httpd...\n")

run(HTTPServer, Server)

INFO:pyngrok.ngrok:Opening tunnel named: http-8585-5f70b61c-89fa-4a65-bbd0-ad4f60946313
INFO:pyngrok.process.ngrok:t=2021-12-02T19:55:35+0000 lvl=info msg="no configuration paths supplied"
INFO:pyngrok.process.ngrok:t=2021-12-02T19:55:35+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml
INFO:pyngrok.process.ngrok:t=2021-12-02T19:55:35+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil
INFO:pyngrok.process.ngrok:t=2021-12-02T19:55:35+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040
INFO:pyngrok.process.ngrok:t=2021-12-02T19:55:35+0000 lvl=info msg="tunnel session started" obj=tunnels.session
INFO:pyngrok.process.ngrok:t=2021-12-02T19:55:35+0000 lvl=info msg="client session established" obj=csess id=766d308af305
INFO:pyngrok.process.ngrok:t=2021-12-02T19:55:35+0000 lvl=info msg=start pg=/api/tunnels id=96fb67d37067bb8b
INFO:pyngrok.process.ngrok:t=2021-12-02T19:55:35+0000 lvl=info msg=end pg=/api/tunnels


##############################################################################################
## Tracking URL: NgrokTunnel: "http://223a-35-193-90-150.ngrok.io" -> "http://localhost:80" ##
##############################################################################################



INFO:pyngrok.process.ngrok:t=2021-12-02T19:55:35+0000 lvl=info msg="started tunnel" obj=tunnels name="http-8585-5f70b61c-89fa-4a65-bbd0-ad4f60946313 (http)" addr=http://localhost:8585 url=http://6f63-35-193-90-150.ngrok.io
INFO:pyngrok.process.ngrok:t=2021-12-02T19:55:35+0000 lvl=info msg="started tunnel" obj=tunnels name=http-8585-5f70b61c-89fa-4a65-bbd0-ad4f60946313 addr=http://localhost:8585 url=https://6f63-35-193-90-150.ngrok.io
INFO:pyngrok.process.ngrok:t=2021-12-02T19:55:35+0000 lvl=info msg=end pg=/api/tunnels id=2739dc4ad93aafb7 status=201 dur=57.782386ms
INFO:pyngrok.process.ngrok:t=2021-12-02T19:55:35+0000 lvl=info msg=start pg="/api/tunnels/http-8585-5f70b61c-89fa-4a65-bbd0-ad4f60946313 (http)" id=db29487b88203629
INFO:root:ngrok tunnel "http://6f63-35-193-90-150.ngrok.io" -> "http://127.0.0.1:8585"
INFO:root:Starting httpd...

INFO:pyngrok.process.ngrok:t=2021-12-02T19:55:35+0000 lvl=info msg=end pg="/api/tunnels/http-8585-5f70b61c-89fa-4a65-bbd0-ad4f60946313 (http)" id=d

[{'position': {'x': 3, 'y': 0.5, 'z': 4}, 'kind': 'Carro', 'colour': 1}, {'position': {'x': 3, 'y': 0.5, 'z': 4}, 'kind': 'Carro', 'colour': 1}, {'position': {'x': 2, 'y': 0.5, 'z': 4}, 'kind': 'Carro', 'colour': 0}, {'position': {'x': 1, 'y': 0.5, 'z': 4}, 'kind': 'Carro', 'colour': 2}, {'position': {'x': -1, 'y': -1, 'z': -1}, 'kind': 'Luz', 'colour': 5}, {'position': {'x': -1, 'y': -1, 'z': -1}, 'kind': 'Luz', 'colour': 1}]


INFO:pyngrok.process.ngrok:t=2021-12-02T19:58:40+0000 lvl=info msg="join connections" obj=join id=f79bbe7c9bc0 l=127.0.0.1:8585 r=[2806:2f0:4000:e5d7:7d2f:f55b:5452:f4bb]:57169
INFO:root:POST request,
Path: /
Headers:
Host: 6f63-35-193-90-150.ngrok.io
User-Agent: UnityPlayer/2021.2.0f1 (UnityWebRequest/1.0, libcurl/7.75.0-DEV)
Content-Length: 94
Accept: */*
Accept-Encoding: deflate, gzip
Content-Type: application/json
X-Forwarded-For: 2806:2f0:4000:e5d7:7d2f:f55b:5452:f4bb
X-Forwarded-Proto: http
X-Unity-Version: 2021.2.0f1



Body:
{"kind": "Carro", "colour": 1, "position": {"x": 3.440000057220459, "y": 0.0, "z": -15.706999778747559}}

127.0.0.1 - - [02/Dec/2021 19:58:40] "POST / HTTP/1.1" 200 -


[{'position': {'x': 6, 'y': 0.5, 'z': 4}, 'kind': 'Carro', 'colour': 1}, {'position': {'x': 6, 'y': 0.5, 'z': 4}, 'kind': 'Carro', 'colour': 1}, {'position': {'x': 5, 'y': 0.5, 'z': 4}, 'kind': 'Carro', 'colour': 0}, {'position': {'x': 4, 'y': 0.5, 'z': 4}, 'kind': 'Carro', 'colour': 2}, {'position': {'x': 3, 'y': 0.5, 'z': 4}, 'kind': 'Carro', 'colour': 2}, {'position': {'x': 2, 'y': 0.5, 'z': 4}, 'kind': 'Carro', 'colour': 0}, {'position': {'x': 1, 'y': 0.5, 'z': 4}, 'kind': 'Carro', 'colour': 1}, {'position': {'x': -1, 'y': -1, 'z': -1}, 'kind': 'Luz', 'colour': 5}, {'position': {'x': -1, 'y': -1, 'z': -1}, 'kind': 'Luz', 'colour': 1}]
